In [1]:
import pandas as pd
import os

In [2]:
train = pd.read_csv('train.csv')
train.head()

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case
0,2,10006,462822612,L,CC,61.0,0,0,0,NaN,0,NaN,29,False
1,2,10006,1459541791,L,MLO,61.0,0,0,0,NaN,0,NaN,29,False
2,2,10006,1864590858,R,MLO,61.0,0,0,0,NaN,0,NaN,29,False
3,2,10006,1874946579,R,CC,61.0,0,0,0,NaN,0,NaN,29,False
4,2,10011,220375232,L,CC,55.0,0,0,0,0.0,0,NaN,21,True


In [3]:
test = pd.read_csv('test.csv')
test.head()

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case
0,2,10048,964141995,L,MLO,62.0,0,0,0,NaN,0,NaN,29,False
1,2,10048,1234933874,L,CC,62.0,0,0,0,NaN,0,NaN,29,False
2,2,10048,1577142909,R,MLO,62.0,0,0,0,NaN,0,NaN,29,False
3,2,10048,1842203124,R,CC,62.0,0,0,0,NaN,0,NaN,29,False
4,2,10050,588678397,L,MLO,67.0,0,0,0,NaN,0,NaN,29,False


In [4]:
def get_filenames(path):
    filenames = set()
    for root, dirs, files in os.walk(path):
        for file in files:
            filenames.add(os.path.splitext(file)[0])
    return filenames

files_low_res = get_filenames('archive')
len(files_low_res)

54706

In [67]:
def check_missing_images(image_ids):
    missing_images = []

    for id in image_ids:
        if str(id) not in files_low_res:
            missing_images.append(id)

    return missing_images

assert (check_missing_images(train['image_id'].values) == []), "Há imagem faltando no conjunto de treino"
assert (check_missing_images(test['image_id'].values) == []), "Há imagem faltando no conjunto de teste"

In [29]:
test.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10962 entries, 0 to 10961
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   site_id                  10962 non-null  int64  
 1   patient_id               10962 non-null  int64  
 2   image_id                 10962 non-null  int64  
 3   laterality               10962 non-null  object 
 4   view                     10962 non-null  object 
 5   age                      10949 non-null  float64
 6   cancer                   10962 non-null  int64  
 7   biopsy                   10962 non-null  int64  
 8   invasive                 10962 non-null  int64  
 9   BIRADS                   5327 non-null   float64
 10  implant                  10962 non-null  int64  
 11  density                  5796 non-null   object 
 12  machine_id               10962 non-null  int64  
 13  difficult_negative_case  10962 non-null  bool   
dtypes: bool(1), float64(2)

In [44]:
def check_dataset(dataset):
    total_cases = len(dataset)
    positive_cases = 0
    
    positive_cases = sum(filter(lambda d: d, dataset['cancer']))
    invasive_cases = sum(filter(lambda d: d, dataset['invasive']))
    missing_birads = len(list(filter(lambda d: pd.isna(d) or d == 0.0, dataset['BIRADS'])))
    missing_density = len(list(filter(lambda d: pd.isna(d), dataset['density'])))
    
    print (f'Total: {total_cases}')
    print (f'Positivos: {positive_cases}, {(positive_cases/total_cases*100):.2f}%')
    print (f'Invasivos: {positive_cases}, {(invasive_cases/total_cases*100):.2f}%')
    print (f'BIRADS faltantes ou incompletos: {missing_birads}, {(missing_birads/total_cases*100):.2f}%')
    print (f'Densidade faltantes: {missing_density}, {(missing_density/total_cases*100):.2f}%')

print ("Conjunto de Treino")
check_dataset(train)
print("\n")
print ("Conjunto de Teste")
check_dataset(test)

Conjunto de Treino
Total: 43744
Positivos: 966, 2.21%
Invasivos: 966, 1.58%
BIRADS faltantes ou incompletos: 29439, 67.30%
Densidade faltantes: 20070, 45.88%


Conjunto de Teste
Total: 10962
Positivos: 192, 1.75%
Invasivos: 192, 1.14%
BIRADS faltantes ou incompletos: 7230, 65.96%
Densidade faltantes: 5166, 47.13%


Considerações:

Quase metade dos dados de BI-RADS[1] ou são incompletos/inconclusivos. Esta feature, com valores entre 1 e 6 são interessantes para a classificação.

[1] BI-RADS (Breast Imaging-Reporting and Data System) é uma ferramenta de avaliação de risco e garantia de qualidade desenvolvida pelo American College of Radiology que fornece um léxico amplamente aceito e um esquema de laudo para imagens da mama. Aplica-se à mamografia, ultrassonografia e ressonância magnética.
Valores possíveis:
 0 - Incompleto, necessária mais imagens. Pouco útil para o trabalho.
 1 - Negativo.
 2 - Benigno.
 3 - Provavelmente benigno.
 4 - Suspeição para maligno.
 5 - Alta suspeição para maligno.
 6 - Maligno.
https://radiopaedia.org/articles/breast-imaging-reporting-and-data-system-bi-rads?lang=us#SnippetTab

Também metade dos dados de densidade estão faltando.

Densidade são classificadas entre A (baixa densidade) e D (alta densidade). Alta densidade apresenta mais risco por possivelmente esconder o cancer das imagens.
https://www.bcna.org.au/breast-health-awareness/mammographic-density-and-screening/

In [51]:
from sklearn.model_selection import train_test_split

# divide o conjunto de treino em treino e validação, e cada um em conjuntos positivados e negativados.
train_df, validation_df = train_test_split(train,
                                           test_size=0.2,
                                           random_state=666,
                                           stratify=train['cancer'])

train_neg_df = train_df[train_df['cancer'] == 0]
train_pos_df = train_df[train_df['cancer'] == 1]

validation_neg_df = validation_df[validation_df['cancer'] == 0]
validation_pos_df = validation_df[validation_df['cancer'] == 1]

# sumário
print(f"Conjunto treino: {len(train_df)}")
print(f"Conjunto validação: {len(validation_df)}")
print("------")
print(f"Conjunto treino: {len(train_neg_df)} casos negativos, {len(train_pos_df)} casos positivos")
print(f"Conjunto validação: {len(validation_neg_df)} casos negativos, {len(validation_pos_df)} casos positivos")

Conjunto treino: 34995
Conjunto validação: 8749
------
Conjunto treino: 34222 casos negativos, 773 casos positivos
Conjunto validação: 8556 casos negativos, 193 casos positivos


In [91]:
import subprocess

path_train = 'train'
path_validation = 'validation'

cmd_clean_dirs = f"rm -fr {path_train} {path_validation}"
cmd_make_dirs = f"mkdir -p {path_train}/positive {path_train}/negative {path_validation}/positive {path_validation}/negative"

# We only live once
subprocess.run(cmd_clean_dirs, shell=True)
subprocess.run(cmd_make_dirs, shell=True)

CompletedProcess(args='mkdir -p train/positive train/negative validation/positive validation/negative', returncode=0)

In [92]:
def copy_images(path, ids, dest):
    for root, dirs, files in os.walk(path):
        for file in files:
            if int(os.path.splitext(file)[0]) in ids:
                subprocess.run(f"cp {root}/{file} {dest}", shell=True)

copy_images('archive', train_neg_df['image_id'].values.tolist(), path_train + '/negative')
copy_images('archive', train_pos_df['image_id'].values.tolist(), path_train + '/positive')
copy_images('archive', validation_neg_df['image_id'].values.tolist(), path_validation + '/negative')
copy_images('archive', validation_pos_df['image_id'].values.tolist(), path_validation + '/positive')
print('Done')

Done


In [93]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255., zoom_range = 0.2)
val_datagen = ImageDataGenerator(rescale = 1./255.)

train_generator = train_datagen.flow_from_directory(
    path_train,
    target_size = (512, 512),
    batch_size = 32,
    class_mode = 'binary')

validation_generator = val_datagen.flow_from_directory(
    path_validation,
    target_size = (512, 512),
    batch_size = 16,
    class_mode = 'binary')

Found 34995 images belonging to 2 classes.
Found 8749 images belonging to 2 classes.


In [96]:
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

base_model = ResNet50(weights = 'imagenet', input_shape = (512, 512, 3), include_top = False)

for layer in base_model.layers:
    layer.trainable = False
    
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = "adam", loss = 'binary_crossentropy', metrics = ["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 16, 16, 2048)      23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 23,850,113
Trainable params: 262,401
Non-trainable params: 23,587,712
______________________________________

In [ ]:
import tensorflow as tf

callback = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", mode = "min", patience = 4)
history = model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 5, epochs = 1, callbacks = callback)

5/5 [==============================] - ETA: 0s - loss: 0.1008 - accuracy: 0.9812 

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
model.save('mammography_pred_model.h5')

In [ ]:
plt.figure(figsize = (15,10))

plt.subplot(2, 2, 1)
plt.plot(accuracy, label = "Training Accuracy")
plt.plot(val_accuracy, label = "Validation Accuracy")
plt.ylim(0.4, 1)
plt.legend(['Train', 'Validation'], loc = 'upper left')
plt.title("Training vs Validation Accuracy")
plt.xlabel('epoch')
plt.ylabel('accuracy')


plt.subplot(2, 2, 2)
plt.plot(loss, label = "Training Loss")
plt.plot(val_loss, label = "Validation Loss")
plt.legend(['Train', 'Validation'], loc = 'upper left')
plt.title("Training vs Validation Loss")
plt.xlabel('epoch')
plt.ylabel('loss')